<a href="https://colab.research.google.com/github/Stefyft/FakenewsRNN/blob/master/Copia_de_Copia_de_Red_Neuronal_Keras_Mejorada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-plot

In [ ]:
#Imports

import nltk
nltk.download("stopwords")
import re
import string
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim import utils
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, BatchNormalization
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import scikitplot.plotters as skplt
from sklearn.metrics import classification_report
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pre-Procesamiento


In [ ]:
# Archivo para el pre-procesamiento

def textClean(text):
    #text = re.sub(r"[^A-Za-z0-9^,!.\/+-=]", " ", text)
    text = re.sub(r"[^\u00f1\u00d1A-ZÀ-ÿa-z0-9^',!.\/+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("spanish"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return (text)


def cleanup(text):
    text = textClean(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text





In [ ]:
data = pd.read_excel("train3.xlsx", sheet_name='train3')
data.head()

,id,title,author,text,label
0,1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",0
1,2,"La palabra ""haiga"", aceptada por la RAE",Hay noticia,"La palabra ""haiga"", aceptada por la RAE La Rea...",0
2,3,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,0
3,4,UNAM capacitará a maestros para aprobar prueba...,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,1
4,5,pretenden aprobar libros escolares con conteni...,Lamula,Alerta: pretenden aprobar libros escolares con...,0


In [ ]:
  missing_rows = []

for i in range(len(data)):
    if data.loc[i, 'text'] != data.loc[i, 'text']:
          missing_rows.append(i)
          data = data.drop(missing_rows).reset_index().drop(['index','id'],axis=1)
data.head()

for i in range(len(data)):
    data.loc[i, 'text'] = cleanup(data.loc[i,'text'])
data.head()


,id,title,author,text,label
0,1,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",El Ruinaversal,rae incluirá palabra lady diccionario idioma e...,0
1,2,"La palabra ""haiga"", aceptada por la RAE",Hay noticia,palabra haiga aceptada rae real academia leng...,0
2,3,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,El Ruinaversal,yordi rosado escribirá diseñará nuevos libros ...,0
3,4,UNAM capacitará a maestros para aprobar prueba...,EL UNIVERSAL,unam capacitará maestros aprobar prueba pisa m...,1
4,5,pretenden aprobar libros escolares con conteni...,Lamula,alerta pretenden aprobar libros escolares cont...,0


In [ ]:
data["text"]

0      rae incluirá palabra lady diccionario idioma e...
1      palabra haiga  aceptada rae real academia leng...
2      yordi rosado escribirá diseñará nuevos libros ...
3      unam capacitará maestros aprobar prueba pisa m...
4      alerta pretenden aprobar libros escolares cont...
                             ...                        
764    faltan detalles ministerio energía recursos na...
765    presidente república lenín moreno anunció vier...
766    sábado 29 agosto instituto microbiología unive...
767    expedición científica logrado marcar decena ti...
768    gobierno volvió reiterar 27 julio flota pesque...
Name: text, Length: 769, dtype: object

In [ ]:
data.shape #769 samples and 5 features

(769, 5)

In [ ]:
def constructLabeledSentences(data):
    sentences = []
    for index, row in data.iteritems():
        sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences
x = constructLabeledSentences(data['text'])

'''sentences = []
for index, row in data['text'].iteritems():
    sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
sentence'''


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  after removing the cwd from sys.path.


"sentences = []\nfor index, row in data['text'].iteritems():\n    sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))\nsentence"

In [ ]:
#x=sentences
x

[LabeledSentence(words=['rae', 'incluirá', 'palabra', 'lady', 'diccionario', 'idioma', 'español', 'definición', 'mujer', 'problemática', 'españa', 'presidente', 'real', 'academia', 'española', 'rae', 'darío', 'villanueva', 'informó', 'conferencia', 'prensa', 'partir', 'próximo', 'mes', 'incluirá', 'término', 'lady', 'nueva', 'palabra', 'diccionario', 'idioma', 'español', 'darío', 'señaló', 'lady', 'servirá', 'definir', 'mujer', 'problemática', 'mujer', 'causa', 'problemas', 'mencionó', 'palabra', 'pocas', 'utilizan', 'idioma', 'inglés', 'castellano', 'significado', 'diferente', 'contadas', 'palabras', 'idioma', 'inglés', 'utilizan', 'español', 'significado', 'globalización', 'personas', 'comenzado', 'adoptar', 'términos', 'anglosajones', 'utilizan', 'significado', 'real', 'embargo', 'caso', 'expresión', 'lady', 'significará', 'mismo', 'idioma', 'original', 'dama', 'inglés', 'sino', 'usará', 'definir', 'mujer', 'problemática', 'acostumbra', 'causar', 'problemas', 'alborotos', 'gente', '

In [ ]:
data['label'].values# separo la columna de falso o no


array([0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,

In [ ]:
data['label'].value_counts(normalize=True)

0    0.50065
1    0.49935
Name: label, dtype: float64

In [ ]:
y= data['label'].values   #Array de 0 y 1 de la columna label

In [ ]:
# Genera vectores para las oraciones 
text_model = Doc2Vec(min_count=1, window=10, vector_size=300, sample=1e-4, negative=5, workers=7, epochs=10, seed=1)
text_model.build_vocab(x) #Vocabulario
text_model.train(x, total_examples=text_model.corpus_count, epochs=text_model.iter)#Entreno
print(text_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


Doc2Vec(dm/m,d300,n5,w10,s0.0001,t7)


In [ ]:
for doc_id in range(len(x)):
  print('Document ({}): «{}»\n'.format(doc_id, ' '.join(x[doc_id].words)))


Document (0): «rae incluirá palabra lady diccionario idioma español definición mujer problemática españa presidente real academia española rae darío villanueva informó conferencia prensa partir próximo mes incluirá término lady nueva palabra diccionario idioma español darío señaló lady servirá definir mujer problemática mujer causa problemas mencionó palabra pocas utilizan idioma inglés castellano significado diferente contadas palabras idioma inglés utilizan español significado globalización personas comenzado adoptar términos anglosajones utilizan significado real embargo caso expresión lady significará mismo idioma original dama inglés sino usará definir mujer problemática acostumbra causar problemas alborotos gente podrá decirle lady fémina cause algún escándalo agresiva provoque algún tipo problema término dejara considerarse palabra exclusiva idioma inglés tras incluida diccionario lengua española pasara ser palabra oficial castellano significado distinto confesó villanueva prese

In [ ]:
train_size = int(0.8 * len(x))
test_size = len(x) - train_size

text_train_arrays = np.zeros((train_size, 300))#lleno de ceros/300
text_test_arrays = np.zeros((test_size, 300))
train_labels = np.zeros(train_size)
test_labels = np.zeros(test_size)

for i in range(train_size):
    text_train_arrays[i] = text_model.docvecs['Text_' + str(i)] #Este objeto docvecs contiene los vectores de párrafo aprendidos de los datos de entrenamiento. Es la etiqueta del final mi identificador en Sentences
    train_labels[i] = y[i]
    print(text_train_arrays[i])


j = 0
for i in range(train_size, train_size + test_size):
    text_test_arrays[j] = text_model.docvecs['Text_' + str(i)]
    test_labels[j] = y[i]
    j = j + 1


Se han truncado las últimas 5000 líneas del flujo de salida.
  0.03464218  0.0170636  -0.20024669  0.23314023  0.05723874  0.01661995
  0.01208666  0.12491249 -0.06270996 -0.00619832 -0.07110769 -0.07358039
  0.05833574 -0.1441637  -0.06496094  0.08026771  0.05383658  0.13046913
  0.16155708  0.14047834  0.01469718 -0.06337978  0.08224836  0.33215696
  0.12978245  0.03527696  0.18129747  0.25065079 -0.07711171  0.00498907
 -0.02901994 -0.17861758  0.06296072  0.01827921 -0.04715804  0.09135042
 -0.14814976  0.08603482 -0.31209585 -0.05972174  0.06906684 -0.05540509
 -0.10179513  0.07381816  0.03294166 -0.26545674  0.13046227 -0.21083738
 -0.10384418 -0.06578667 -0.04635957  0.00510225 -0.10008661  0.08462986
 -0.23933674 -0.25010505  0.01141552 -0.05217279  0.22133936  0.22604302
 -0.30194876  0.26345322  0.04303373  0.05241047  0.06809849 -0.0590287
 -0.12244981  0.22344178  0.1225665  -0.0526236   0.2063475   0.03151843
  0.26343283 -0.17818125 -0.20757502  0.24309903 -0.0674848   0.

In [ ]:


  
# Extracción de características y pre-procesamiento

xtr,xte,ytr,yte = text_train_arrays, text_test_arrays, train_labels, test_labels
np.save('./xtr', xtr)
np.save('./xte', xte)
np.save('./ytr', ytr)
np.save('./yte', yte)


# Carga de vectores de características

xtr = np.load('./xtr.npy')
print(xtr)
xte = np.load('./xte.npy')
print(xte)
ytr = np.load('./ytr.npy')
print(ytr)
yte = np.load('./yte.npy')
print(yte)

[[ 4.21413109e-02 -4.36812975e-02 -2.38964078e-03 ... -3.78668588e-03
   3.99649255e-02 -1.93289310e-01]
 [ 2.39455905e-02 -2.19126027e-02 -2.61679664e-03 ... -5.22474467e-04
   2.04310846e-02 -1.02610804e-01]
 [ 4.51845489e-02 -4.54657041e-02 -3.80124408e-03 ... -5.51955681e-03
   4.32226770e-02 -2.12622181e-01]
 ...
 [ 2.17505079e-02 -2.42842510e-02 -8.16843705e-04 ... -3.82753700e-04
   2.24366896e-02 -1.04487605e-01]
 [ 1.64899342e-02 -1.89827941e-02 -2.08667683e-04 ... -1.19644334e-03
   1.68920383e-02 -8.02698955e-02]
 [ 3.62499952e-02 -3.58420797e-02 -1.96224172e-03 ... -4.03383235e-03
   3.40157524e-02 -1.69890672e-01]]
[[ 1.64940953e-02 -1.56585574e-02  6.80501398e-04 ... -2.55574682e-03
   1.30365491e-02 -7.36283064e-02]
 [ 7.59602524e-03 -1.10672656e-02  4.15319722e-04 ... -1.58490916e-03
   8.16451758e-03 -4.25417684e-02]
 [ 2.65943501e-02 -3.04355826e-02  2.74317135e-04 ... -2.03328463e-03
   2.73798779e-02 -1.30448237e-01]
 ...
 [ 1.31254131e-02 -1.37025798e-02 -1.0462800

In [ ]:
# Modelo de la red neuronal

def baseline_model():
    model = Sequential()
    model.add(Dense(256, input_dim=300, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    #model.add(Dense(80, activation='relu', kernel_initializer='normal'))
    model.add(Dense(2, activation="softmax"))

    # Gradiente descendiente
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    # Configuración del proceso de aprendizaje del modelo
    #model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    #return model
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

# Entrenamiento del modelo

model = baseline_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               77056     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 143,362
Trainable params: 143,362
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(xtr, ytr, test_size=0.05, random_state=10)


In [ ]:
x_train.shape


(584, 300)

In [ ]:
x_test.shape


(31, 300)

In [ ]:

y_train.shape


(584,)

In [ ]:

y_test.shape

(31,)

In [ ]:
#Entrenamiento del modelo
#x_train, x_test, y_train, y_test = train_test_split(xtr, ytr, test_size=0.05, random_state=10)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y = np_utils.to_categorical((label_encoder.transform(y_train)))
label_encoder.fit(y_test)
encoded_y_test = np_utils.to_categorical((label_encoder.transform(y_test)))


estimator = model.fit(x_train, encoded_y, epochs=40, batch_size=64)#Data del train
print("Model Trained!")
estimator.history


Epoch 1/40
10/10 [==============================] - 0s 5ms/step - loss: 0.6908 - accuracy: 0.5240
Epoch 2/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6904 - accuracy: 0.5000
Epoch 3/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6811 - accuracy: 0.5274
Epoch 4/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6754 - accuracy: 0.5702
Epoch 5/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6752 - accuracy: 0.5479
Epoch 6/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6709 - accuracy: 0.5856
Epoch 7/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6729 - accuracy: 0.5771
Epoch 8/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6645 - accuracy: 0.5856
Epoch 9/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6752 - accuracy: 0.5771
Epoch 10/40
10/10 [==============================] - 0s 3ms/step - loss: 0.6671 - accuracy: 0.5599
Epoch 11/40
10/10 [

{'accuracy': [0.5239726305007935,
  0.5,
  0.5273972749710083,
  0.5702054500579834,
  0.5479452013969421,
  0.585616409778595,
  0.5770547986030579,
  0.585616409778595,
  0.5770547986030579,
  0.5599315166473389,
  0.5804794430732727,
  0.5804794430732727,
  0.5890411138534546,
  0.5890411138534546,
  0.5941780805587769,
  0.6027397513389587,
  0.5873287916183472,
  0.585616409778595,
  0.5787671208381653,
  0.5890411138534546,
  0.6113013625144958,
  0.5941780805587769,
  0.6027397513389587,
  0.5873287916183472,
  0.5924657583236694,
  0.5976027250289917,
  0.5976027250289917,
  0.573630154132843,
  0.5924657583236694,
  0.5993150472640991,
  0.5976027250289917,
  0.5873287916183472,
  0.585616409778595,
  0.5821917653083801,
  0.590753436088562,
  0.6010273694992065,
  0.5976027250289917,
  0.5839040875434875,
  0.5958904027938843,
  0.585616409778595],
 'loss': [0.6908052563667297,
  0.690357506275177,
  0.6811279654502869,
  0.67542564868927,
  0.6751918196678162,
  0.6709429025

In [ ]:
#Evaluar el modelo en los datos de prueba
#Estos valores indican qué tan bien o mal se comporta nuestro modelo con los nuevos datos que nunca ha visto
score = model.evaluate(x_test, encoded_y_test)
print("test loss, test acc:", score)

1/1 [==============================] - 0s 1ms/step - loss: 0.5853 - accuracy: 0.7742
test loss, test acc: [0.5853350162506104, 0.774193525314331]


In [ ]:
# El modelo clasifica el %  correctamente de los datos que nunca había visto antes.
print("Accuracy = " + format(score[1]*100, '.2f') + "%")

Accuracy = 77.42%


In [ ]:
noticia=[""]
words = nltk.word_tokenize(noticia)
print(words)

In [ ]:
#Generar predicciones
#print(x_test[10])
prediction=model.predict(x_test) #The predict () método devuelve un vector con las predicciones para los elementos del conjunto de datos completo.
#for i in range(len(x_test)):
#    print("X=%s, Predicted=%s" % (x_test[i], prediction[i]))
#print("X=%s, Predicted=%s" % (x_test[10], prediction[10]))

new_t = x_test[:20]
#print(ytr)
prediction2=model.predict_classes(new_t)
print(x_test[10])
#np.array(xte)[prediction2]

#print(prediction[10])
#np.argmax(prediction[10])

In [ ]:
#Plot
probabs = model.predict_proba(x_test)
y_pred = np.argmax(probabs, axis=1)


#Plot de matriz de confusión
skplt.plot_confusion_matrix(y_test, y_pred)
plt.show()
 

# Prueba
df = pd.DataFrame(estimator.history)
df.head()
df.plot(figsize=(8, 5))
